In [4]:
import mdf_iter               # MDF4 데이터 불러오는 라이브러리
import canedge_browser        # 로컬 또는 S3에 저장된 데이터 가져오는 라이브러리
import can_decoder            # 데이터를 복호화하는 라이브러리
import numpy as np
import pandas as pd           # 데이터분석 라이브러리
from datetime import datetime, timezone  # 날짜와 시간 데이터를 가져올 수 있는 파이썬 라이브러리
from utils import setup_fs, load_dbc_files, restructure_data, add_custom_sig, ProcessData # 폴더 내 위치한 Custom 라이브러리

In [5]:
# specify devices to process (from local/S3), DBC files and start time
# devices = ["LOG/test"]
# dbc_paths = ["dbc_files/CSS-Electronics-OBD2-v1.4.dbc"]
# s3 또는 로컬에 저장한 파일을 불러옵니다. 
# 만약 s3에 저장한 경우라면, s3=True를 입력합니다.

# 시작시간
start = datetime(year=2020, month=1, day=13, hour=0, tzinfo=timezone.utc)

In [6]:
#  load, DBC decode and print MF4 log files

# devices MF4 파일을 넣는곳
devices = ["LOG/958D2219"]
# dbc_paths dbc 파일을 넣는곳

dbc_paths = ["dbc_files/CSS-Electronics-OBD2-v1.4.dbc"]
# devices = ["LOG/test"]
# dbc_paths = ["dbc_files/tp_j1939.dbc"]

db_list = load_dbc_files(dbc_paths)
fs = setup_fs(s3=False, key="", secret="", endpoint="")

log_files = canedge_browser.get_log_files(fs, devices)

print(db_list)
print(log_files)
proc = ProcessData(fs, db_list)

df_phys_all = pd.DataFrame()


# for log_file in log_files:
#    df_raw,df_id = proc.get_raw_data(log_file)
#    df_phys = proc.extract_phys(df_raw)
#    print(df_phys)


[]


In [7]:
# setup filesystem (local/S3), load DBC files and list log files for processing
fs = setup_fs(s3=False, key="", secret="", endpoint="")


In [8]:
# 리스트형태로 입력한 dbc 파일경로를 받아 list를 생성합니다
db_list = load_dbc_files(dbc_paths)
# canedge_browser.get_log_files는 조건에 맞는 로그 파일 경로를 List로 반환합니다.
log_files = canedge_browser.get_log_files(fs, devices)
print(f"Found a total of {len(log_files)} log files")

print(db_list)
print(log_files)


Found a total of 0 log files
[]


In [9]:
proc = ProcessData(fs, db_list, signals=[])

# 데이터 프레임을 먼저만든뒤
df_phys_all = pd.DataFrame()


In [10]:
# 로그파일 리스트의 데이터를 불러와 데이터프레임에 append
for log_file in log_files:
    # .get_raw_data 는 df_raw 값과 device_id 값 리턴
    df_raw,device_id = proc.get_raw_data(log_file)
    df_phys = proc.extract_phys(df_raw)
    proc.print_log_summary(device_id,log_file,df_phys)
    df_phys_all = df_phys_all.append(df_phys)



In [11]:
# df.info 는 함수의 전체정보를 요약해서 출력합니다.
df_phys_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

In [12]:
df_phys_all.head()

""


In [13]:
# def ratio(s1,s2):
#     return s2/s1 if s1 else np.nan

In [14]:
# df_phys_all = add_custom_sig(df_phys_all, "WheelBasedVehicleSpeed", "EngineSpeed", ratio, "RatioRpmSpeed")

In [15]:
df_phys_all.tail()

""


In [16]:
# 위의 데이터프레임을 재구조화 후 csv 파일로내보내기
df_phys_join = restructure_data(df_phys=df_phys_all, res="1S")
df_phys_join
# df_phys_join.to_csv("output_joined.csv")



,TimeStamp


In [17]:
# 엔진속도를 시각화 
import matplotlib.pyplot as plt

plt.plot(df_phys_join["S1_PID_0C_EngineRPM"])
plt.xlabel("Time")
plt.ylabel("EngineRPM")
plt.legend("E")
plt.show()

KeyError: 'S1_PID_0C_EngineRPM'